In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import rnacomp
import rnacomp.models
from rnacomp.fastai_fit import *
from rnacomp.dataset import LenMatchBatchSampler, DeviceDataLoader
from rnacomp.utils import seed_everything, MAE, loss_laplace
import gc

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import wandb


In [4]:
class CFG:
    path = Path("../data/")
    pathbb = Path("../data/Ribonanza_bpp_files")
    split_id = Path('../eda/fold_split.csv')
    bs = 64
    num_workers = 12
    device = 'cuda'
    seed = 2023
    out = 'exp_07'
    dataset_name = 'RNA_DatasetBaselineSplitbppV0'
    
    model_name = 'RNA_ModelV6'
    dim = 192
    depth = 12
    dim_head = 32
   
    epoch = 64
    lr = 5e-4
    wd = 0.05
    pct_start = 0.02
    
    
seed_everything(CFG.seed)
os.makedirs(CFG.out, exist_ok=True)

In [5]:
fns = list(CFG.pathbb.rglob("*.txt"))
bpp_df = pd.DataFrame({"bpp": fns})
bpp_df['sequence_id'] = bpp_df['bpp'].apply(lambda x: x.stem)
df = pd.read_parquet(CFG.path/'train_data.parquet')
split = pd.read_csv(CFG.split_id)
df = pd.merge(df, split, on='sequence_id')
df = pd.merge(df, bpp_df, on='sequence_id')
df_train = df.query('is_train==True').reset_index(drop=True)
df_valid = df.query('is_train==False').reset_index(drop=True)

ds_train = getattr(rnacomp.dataset, CFG.dataset_name)(df_train, mode='train',sn_train=False)
ds_train_len = getattr(rnacomp.dataset, CFG.dataset_name)(df_train, mode='train', mask_only=True,sn_train=False)
sampler_train = torch.utils.data.RandomSampler(ds_train_len)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=CFG.bs,
            drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=CFG.num_workers,
            persistent_workers=True), CFG.device)


ds_val = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid, mode='eval')
ds_val_len = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid, mode='eval', mask_only=True)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
               drop_last=False)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
               batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)

data = DataLoaders(dl_train,dl_val)
gc.collect()

16

In [ ]:
learn = Learner(data,
                getattr(rnacomp.models, CFG.model_name)().cuda(), 
                path = CFG.out, 
                loss_func=loss_laplace,
                cbs=[GradientClip(3.0),
                    CSVLogger(),
                    SaveModelCallback(monitor='mae',comp=np.less,at_end=True)],
                metrics=[MAE()]).to_fp16() 
learn.fit_one_cycle(CFG.epoch, lr_max=CFG.lr, wd=CFG.wd, pct_start=CFG.pct_start)

epoch,train_loss,valid_loss,mae,time
0,0.135874,0.154494,0.165600,25:41
1,0.128311,0.148929,0.160252,25:46
2,0.126219,0.144770,0.155648,25:59
3,0.124412,0.143524,0.154403,26:01
4,0.123816,0.142347,0.153098,26:03
5,0.124092,0.140836,0.151494,26:02
6,0.122731,0.141441,0.152299,26:04
7,0.122769,0.140631,0.151413,26:07
8,0.121953,0.138661,0.149238,26:06
9,0.122030,0.138792,0.149358,26:05


Better model found at epoch 0 with mae value: 0.16560026330254604.
Better model found at epoch 1 with mae value: 0.16025246294606152.
Better model found at epoch 2 with mae value: 0.1556480794234897.
Better model found at epoch 3 with mae value: 0.15440307034267695.
Better model found at epoch 4 with mae value: 0.15309758926475522.
Better model found at epoch 5 with mae value: 0.15149446187937807.
Better model found at epoch 7 with mae value: 0.15141328586263497.
Better model found at epoch 8 with mae value: 0.1492376458565687.
Better model found at epoch 10 with mae value: 0.14858953481104872.


In [ ]:
res = dict()
for l in df_valid["L"].unique():
    ds_val = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid.query("L==@l").copy(), mode='eval')
    ds_val_len = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid.query("L==@l").copy(), mode='eval', mask_only=True)
    sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
    len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
                   drop_last=False)
    dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
                   batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)

    data = DataLoaders(dl_val,dl_val)
    learn = Learner(data,
                    getattr(rnacomp.models, CFG.model_name)().cuda(), 
                    path = CFG.out, 
                    loss_func=loss_laplace,
                    metrics=[MAE()]).to_fp16() 
    learn.load('model')
    learn.eval()
    loss_, score_ = learn.validate()
    res[l]= score_

res = pd.DataFrame(pd.Series(res)).reset_index()
res.columns = ['L', 'mae']
res.sort_values(by="L")